In [2]:
df_p_f = pd.read_csv('/kaggle/input/fakenewsnet/full_dataset/politifact_fake.csv')
df_p_r = pd.read_csv('/kaggle/input/fakenewsnet/full_dataset/politifact_real.csv')
df_g_f = pd.read_csv('/kaggle/input/fakenewsnet/full_dataset/gossipcop_fake.csv')
df_g_rs = [pd.read_csv(f'/kaggle/input/fakenewsnet/full_dataset/gossipcop_real_{i}.csv') for i in range(1,5)]
df_g_r = pd.concat(df_g_rs,ignore_index = True)

In [3]:
df_p_f.head()

,id,news_url,title,tweet_ids,text,baseline
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,NaN,NaN
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,NaN,NaN
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...,NaN,NaN
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,in Sports Oscar Pistorius Attempts To Commit S...,in Sports Oscar Pistorius Attempts To Commit S...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,NaN,NaN


In [4]:
df_p_f[df_p_f['news_url'].isna()]

,id,news_url,title,tweet_ids,text,baseline
244,politifact14427,NaN,Friendly Fire … Charlottesville Car attacker ...,896524918619328512\t896559582583107585\t896657...,NaN,NaN
296,politifact13724,NaN,Breaking President Trump makes English the of...,797669308532924416\t824634342534107147\t824648...,NaN,NaN
334,politifact15294,NaN,Post claiming free tickets on Allegiant Air,NaN,NaN,NaN
352,politifact15242,NaN,Actress Emma Stone ‘For the first time in his...,939625499629899776\t939626793442664448\t939730...,NaN,NaN


In [5]:
import numpy as np
def parse_url(url):
    if url != url:
        return np.nan
    id1 = url.find('//')
    if id1>=0:
        url = url[id1+2:]
    id2 = url.find('www.')
    if id2>=0:
        url = url[id2+4:]
    id3 = url.find('/')
    url = url[:id3]
    return url

In [6]:
df_p_f['parsed_url'] = df_p_f['news_url'].apply(lambda x: parse_url(x))
df_p_f['label'] = 0
df_p_r['parsed_url'] = df_p_r['news_url'].apply(lambda x: parse_url(x))
df_p_r['label'] = 1
df_g_f['parsed_url'] = df_g_f['news_url'].apply(lambda x: parse_url(x))
df_g_f['label'] = 0
df_g_r['parsed_url'] = df_g_r['news_url'].apply(lambda x: parse_url(x))
df_g_r['label'] = 1

In [7]:
df_g_r.head()

,id,news_url,title,tweet_ids,text,baseline,parsed_url,label
0,gossipcop-882573,https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,912371411146149888\t912371528343408641\t912372...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,brides.com,1
1,gossipcop-875924,https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner refusing to discuss Tyga on Life ...,901989917546426369\t901989992074969089\t901990...,""",",""",",dailymail.co.uk,1
2,gossipcop-894416,https://en.wikipedia.org/wiki/Quinn_Perkins,Quinn Perkins,931263637246881792\t931265332022579201\t931265...,"Quinn Perkins, formerly Lindsay Dwyer, is a fi...","Quinn Perkins, formerly Lindsay Dwyer, is a fi...",en.wikipedia.org,1
3,gossipcop-857248,https://www.refinery29.com/en-us/2018/03/19192...,I Tried Kim Kardashian's Butt Workout & Am For...,868114761723936769\t868122567910936576\t868128...,Story from FitnessI Tried Kim Kardashian's But...,Story from FitnessI Tried Kim Kardashian's But...,refinery29.com,1
4,gossipcop-884684,https://www.cnn.com/2017/10/04/entertainment/c...,Celine Dion donates concert proceeds to Vegas ...,915528047004209152\t915529285171122176\t915530...,An emotional Celine Dion returned to the stage...,An emotional Celine Dion returned to the stage...,cnn.com,1


In [8]:
everything_df = pd.concat([df_g_f,df_g_r,df_p_f,df_p_r],ignore_index = True)
everything_df.dropna(subset = ['parsed_url'], inplace=True)

In [9]:
len(everything_df)

22866

In [10]:
set_true = list(set(everything_df[everything_df['label']==1]['parsed_url']))
set_false = list(set(everything_df[everything_df['label']==0]['parsed_url']))

In [11]:
middle = 0
for site in set_true:
    if site in set_false:
        middle += 1
        
        
len(set_true)-middle, middle, len(set_false)-middle

(1511, 425, 553)

In [12]:
everything_df[(everything_df['parsed_url'].isin(set_true)) & (~everything_df['parsed_url'].isin(set_false))]

,id,news_url,title,tweet_ids,text,baseline,parsed_url,label
5328,gossipcop-899309,https://tech2.org/chris-evans-millie-bobby-bro...,"Chris Evans, Millie Bobby Brown, Snoop Dogg an...",940013737204899841\t940014043619831808\t940014...,NaN,NaN,tech2.org,1
5331,gossipcop-926957,https://decider.com/2018/06/11/claws-season-2-...,When Will ‘Claws’ Season 2 Be On Hulu?,984486548996419584\t984490492787351556\t984491...,Share this:Click to share on Facebook (Opens i...,Share this:Click to share on Facebook (Opens i...,decider.com,1
5332,gossipcop-930832,http://www.criticschoice.com/critics-choice-aw...,Critics' Choice Awards - Critics' Choice Awards,989840869443457024\t990987021194743808,NaN,NaN,criticschoice.com,1
5339,gossipcop-915349,https://www.vibe.com/2018/02/rihanna-birthday-...,Rihanna Rings In Her 30th Birthday With Toni B...,966403439688388613\t966404241962164225\t966404...,From red carpets and special events to everyda...,From red carpets and special events to everyda...,vibe.com,1
5348,gossipcop-911983,http://celebritymaximum.com/2018/02/06/so-long...,Celebritymaximum.com,960957805803970560\t960958553400885248\t960958...,NaN,NaN,celebritymaximum.com,1
...,...,...,...,...,...,...,...,...
23185,politifact8071,http://www.whitehouse.gov/the-press-office/201...,Remarks by the President on the Economy -- Kno...,359704596446330880\t360079198183886848\t360082...,Jump to main content Jump to navigation Knox C...,Jump to main content Jump to navigation Knox C...,whitehouse.gov,1
23186,politifact8152,http://www.thedailyshow.com/full-episodes/mon-...,The Daily Show with Trevor Noah,971170110\t1321359151\t1322290944\t1379752989\...,"Desi Lydic joined ""The Daily Show"" as a corres...","Desi Lydic joined ""The Daily Show"" as a corres...",thedailyshow.com,1
23188,politifact1204,http://select.nytimes.com/gst/abstract.html?re...,IMPEACHMENT HINT STIRS HOUSE CLASH; Democratic...,NaN,AdvertisementSKIP ADVERTISEMENTSupported bySKI...,NaN,select.nytimes.com,1
23191,politifact14731,https://www.flake.senate.gov/public/index.cfm/...,Flake: “Religious tests should have no place i...,NaN,NaN,NaN,flake.senate.gov,1


In [13]:
len(everything_df[(everything_df['parsed_url'].isin(set_true)) & (everything_df['parsed_url'].isin(set_false))])

17200

In [14]:
len(everything_df[(~everything_df['parsed_url'].isin(set_true)) & (everything_df['parsed_url'].isin(set_false))])

1182

In [83]:
to_classify = everything_df[(everything_df['parsed_url'].isin(set_true)) & (everything_df['parsed_url'].isin(set_false))]

In [84]:
to_classify['tweets'] = to_classify['tweet_ids'].apply(lambda x: len(x.split('\t')) if x==x else 0)
to_classify.head()

,id,news_url,title,tweet_ids,text,baseline,parsed_url,label,tweets
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,Miley Cyrus and Liam Hemsworth reportedly tied...,Miley Cyrus and Liam Hemsworth reportedly tied...,dailymail.co.uk,0,97
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,NaN,NaN,hollywoodlife.com,0,15
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,Thousands are taking the streets to protest Pr...,Thousands are taking the streets to protest Pr...,variety.com,0,86
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,"On Thursday, the 14-year-old shared a picture ...","On Thursday, the 14-year-old shared a picture ...",dailymail.co.uk,0,14
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,Oscar nominations for the 90th annual awards w...,Oscar nominations for the 90th annual awards w...,variety.com,0,66


# Text preprocessing

In [18]:
import re
import nltk
import spacy
import string
from nltk.corpus import stopwords
nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/co

In [24]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
PUNCT_TO_REMOVE = string.punctuation
STOPWORDS = set(stopwords.words('english'))
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

def preprocess (text):
    def lower(text):
        return text.lower()
    def remove_punctuation(text):
        return text.translate(text.maketrans('', '', PUNCT_TO_REMOVE))
    def remove_stopwords(text):
        return " ".join([word for word in str(text).split() if word not in STOPWORDS])
    def stem_words(text):
        return " ".join([stemmer.stem(word) for word in text.split()])
    def lemmatize_words(text):
        pos_tagged_text = nltk.pos_tag(text.split())
        return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
    return lemmatize_words(stem_words(remove_stopwords(remove_punctuation(lower(text)))))
        

In [95]:
to_classify['title_process'] = to_classify['title'].apply(lambda x: preprocess(x))

In [96]:
to_classify.head()

,id,news_url,title,tweet_ids,text,baseline,parsed_url,label,tweets,title_process
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,Miley Cyrus and Liam Hemsworth reportedly tied...,Miley Cyrus and Liam Hemsworth reportedly tied...,dailymail.co.uk,0,97,miley cyru liam hemsworth secretli get marri
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,NaN,NaN,hollywoodlife.com,0,15,pari jackson cara delevingn enjoy night match ...
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,Thousands are taking the streets to protest Pr...,Thousands are taking the streets to protest Pr...,variety.com,0,86,celebr join tax march protest donald trump
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,"On Thursday, the 14-year-old shared a picture ...","On Thursday, the 14-year-old shared a picture ...",dailymail.co.uk,0,14,cindi crawford daughter kaia gerber wear wig d...
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,Oscar nominations for the 90th annual awards w...,Oscar nominations for the 90th annual awards w...,variety.com,0,66,full list 2018 oscar nomin – varieti


In [97]:
y = to_classify['label']
x = to_classify['title_process']

In [101]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

tfidf_vect = TfidfVectorizer(use_idf = True,stop_words = 'english')
tfidf_train = tfidf_vect.fit_transform(x_train)
tfidf_test = tfidf_vect.transform(x_test)
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vect.get_feature_names_out())

In [61]:
Adab = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10),n_estimators=5,random_state=1)
Adab.fit(tfidf_train, y_train)
y_pred3 = Adab.predict(tfidf_test)
ABscore = metrics.accuracy_score(y_test,y_pred3)
print("accuracy: %0.3f" %ABscore)

accuracy: 0.782


In [102]:
rfc = RandomForestClassifier(n_estimators=100,random_state=1)
rfc.fit(tfidf_train, y_train)
y_pred = rfc.predict(tfidf_test)
RFscore = metrics.accuracy_score(y_test, y_pred)
print("accuracy: %0.3f" %RFscore)

accuracy: 0.828


4484, 17200, 1182

In [103]:
sum((y_pred==y_test) & (y_pred==1)), sum((y_pred==y_test) & (y_pred==0)), sum((y_pred!=y_test) & (y_pred==1)), sum((y_pred!=y_test) & (y_pred==0))

(2415, 434, 427, 164)

Test 20%, AdaBoost 0.810,RFC 0.830

50%, AdaBoost 0.797,RFC 0.821

80%, AdaBoost 0.782,RFC 0.810

(9636, 1504, 1923, 697)


# baseline

In [85]:
to_classify_2 = to_classify[~to_classify['baseline'].isna()]
to_classify_2['baseline_process'] = to_classify_2['baseline'].apply(lambda x: preprocess(x))
y = to_classify_2['label']
x = to_classify_2['baseline_process']



In [92]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

tfidf_vect = TfidfVectorizer(use_idf = True,stop_words = 'english')
tfidf_train = tfidf_vect.fit_transform(x_train)
tfidf_test = tfidf_vect.transform(x_test)
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vect.get_feature_names_out())

In [86]:
Adab = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10),n_estimators=5,random_state=1)
Adab.fit(tfidf_train, y_train)
y_pred3 = Adab.predict(tfidf_test)
ABscore = metrics.accuracy_score(y_test,y_pred3)
print("accuracy: %0.3f" %ABscore)

accuracy: 0.740


In [93]:
rfc = RandomForestClassifier(n_estimators=100,random_state=1)
rfc.fit(tfidf_train, y_train)
y_pred = rfc.predict(tfidf_test)
RFscore = metrics.accuracy_score(y_test, y_pred)
print("accuracy: %0.3f" %RFscore)

accuracy: 0.832


In [94]:
sum((y_pred==y_test) & (y_pred==1)), sum((y_pred==y_test) & (y_pred==0)), sum((y_pred!=y_test) & (y_pred==1)), sum((y_pred!=y_test) & (y_pred==0))

(2276, 244, 477, 31)

# text

In [106]:
to_classify_3 = to_classify[~to_classify['text'].isna()]
to_classify_3['text_process'] = to_classify_3['text'].apply(lambda x: preprocess(x))
y = to_classify_3['label']
x = to_classify_3['text_process']
len(to_classify_3)

15199

In [113]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.80)

tfidf_vect = TfidfVectorizer(use_idf = True,stop_words = 'english')
tfidf_train = tfidf_vect.fit_transform(x_train)
tfidf_test = tfidf_vect.transform(x_test)
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vect.get_feature_names_out())

In [114]:
rfc = RandomForestClassifier(n_estimators=100,random_state=1)
rfc.fit(tfidf_train, y_train)
y_pred = rfc.predict(tfidf_test)
RFscore = metrics.accuracy_score(y_test, y_pred)
print("accuracy: %0.3f" %RFscore)

accuracy: 0.815


In [115]:
sum((y_pred==y_test) & (y_pred==1)), sum((y_pred==y_test) & (y_pred==0)), sum((y_pred!=y_test) & (y_pred==1)), sum((y_pred!=y_test) & (y_pred==0))

(9173, 738, 2158, 91)

# WordCloud

In [ ]:
from wordcloud import WordCloud 
wc = WordCloud(background_color="white", max_words=100,
               max_font_size=256,
               random_state=42, width=2000, height=1000)
wc.generate(' '.join(everything_df[~everything_df['text'].isna()]['text']))
plt.imshow(wc)
plt.axis('off')
plt.show()